In [61]:
import json
import numpy as np
import os
import pandas as pd
import urllib2
from sklearn.preprocessing import StandardScaler
import h5py

In [62]:
coins = ['BTC', 'LTC', 'ETH', 'XMR']
df_list=[]
for coin in coins:
    url = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_'+coin+'&start=1439014500&end=9999999999&period=300'
    openUrl = urllib2.urlopen(url)
    r = openUrl.read()
    openUrl.close()
    d = json.loads(r.decode())
    df = pd.DataFrame(d)
    original_columns=[u'close', u'date', u'high', u'low', u'open']
    new_columns = ['Close','Timestamp','High','Low','Open']
    df = df.loc[:,original_columns]
    df.columns = new_columns
    df_list.append(df)

In [63]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [70]:
superviseds= None
supervised_timestamp_Xs =None
supervised_timestamp_Ys = None
scaled_Xs = None
scaled_Ys = None
Xs = None
Ys = None
first_it=True
for df in df_list:    
    columns = ['Close']
    original_df = df.copy()
    time_stamps = df['Timestamp']
    df = df.loc[:,columns]
    
    
    supervised = series_to_supervised(df,256,16)
    supervised = np.array(supervised)
    supervised_timestamp = series_to_supervised(time_stamps[:,None],256,16)
    supervised_timestamp = np.array(supervised_timestamp)
    
    
    
    # standardize the data
    scaler = StandardScaler()
    scaled_supervised=[]
    scaler.fit(df)
    for i in range(len(supervised)):
        scaled_supervised.append(scaler.transform(supervised[i].reshape(-1,1)))
    scaled_supervised = np.array(scaled_supervised)
    
    
    # change them to 3 dimensional
    supervised = supervised[:,:,None]
    supervised_timestamp = supervised_timestamp[:,:,None]
    
    #splitting into training X and testing Y
    X = supervised[:,:256]
    Y = supervised[:,256:]
    scaled_X = scaled_supervised[:,:256]
    scaled_Y = scaled_supervised[:,256:]
    supervised_timestamp_X = supervised_timestamp[:,:256]
    supervised_timestamp_Y = supervised_timestamp[:,256:]
#     print scaled_X.shape
#     print scaled_Y.shape
#     print supervised_timestamp_X.shape
#     print supervised_timestamp_Y.shape
#     print supervised.shape
    
    if first_it:
        Xs = X
        Ys = Y
        scaled_Xs = scaled_X
        scaled_Ys = scaled_Y
        supervised_timestamp_Xs = supervised_timestamp_X
        supervised_timestamp_Ys = supervised_timestamp_Y
        superviseds = supervised
        first_it=False
    else:
        #stack on the third dimension
        scaled_Xs = np.concatenate((scaled_Xs,scaled_X), axis=2)
        #scaled_Ys = np.concatenate((scaled_Ys,scaled_Y), axis=2)
        supervised_timestamp_Xs = np.concatenate((supervised_timestamp_Xs,supervised_timestamp_X), axis=2)
        
        superviseds = np.concatenate((superviseds,supervised), axis=2)
        Xs = np.concatenate((Xs,X), axis=2)
        

In [71]:
print superviseds.shape
print supervised_timestamp_Xs.shape
print supervised_timestamp_Ys.shape
print scaled_Xs.shape
print scaled_Ys.shape
print Xs.shape
print Ys.shape

(253180, 272, 4)
(253180, 256, 4)
(253180, 16, 1)
(253180, 256, 4)
(253180, 16, 1)
(253180, 256, 4)
(253180, 16, 1)


In [72]:
#assume 8000 timestemps a month
nb_months = Xs.shape[0]/8000
nb_months

31

In [73]:
inputs=[]
outputs=[]
# input_times=[]
# output_times=[]
# original_inputs=[]
# original_outputs=[]
for i in range(nb_months-5):
#     original_inputs.append(Xs[i*8000:(i+5)*8000,:,:])
#     original_outputs.append(Ys[i*8000:(i+5)*8000,:,:])
    inputs.append(scaled_Xs[i*8000:(i+5)*8000,:,:])
    outputs.append(scaled_Ys[i*8000:(i+5)*8000,:,:])
#     input_times.append(supervised_timestamp_Xs[i*8000:(i+5)*8000,:,:])
#     output_times.append(supervised_timestamp_Ys[i*8000:(i+5)*8000,:,:])
# original_outputs = np.array(original_outputs)
# original_inputs = np.array(original_inputs)
inputs = np.array(inputs)
outputs= np.array(outputs)
input_times = np.array(input_times)
output_times = np.array(output_times)

In [74]:
test_size = Xs.shape[0] % 8000

In [75]:
original_test_inputs = Xs[-test_size:,:,:]
original_test_outputs = Ys[-test_size:,:,:]
test_inputs = scaled_Xs[-test_size:,:,:]
test_outputs = scaled_Ys[-test_size:,:,:]
test_input_times = supervised_timestamp_Xs[-test_size:,:,:]
test_output_times = supervised_timestamp_Ys[-test_size:,:,:]
original_datas = superviseds[-(len(original_test_outputs)):,:,0][:,:,None]

In [76]:
print test_output_times.shape
print test_input_times.shape
print original_test_outputs.shape
print original_test_inputs.shape
print test_inputs.shape
print test_outputs.shape
print inputs.shape
print outputs.shape
print original_datas.shape

(5180, 16, 1)
(5180, 256, 4)
(5180, 16, 1)
(5180, 256, 4)
(5180, 256, 4)
(5180, 16, 1)
(26, 40000, 256, 4)
(26, 40000, 16, 1)
(5180, 272, 1)


In [77]:
file_name ='data/allcoin2015to2017_wf.h5'

In [78]:
import h5py
with h5py.File(file_name, 'w') as f:
    f.create_dataset("inputs", data = inputs )
    f.create_dataset('outputs', data = outputs)
    f.create_dataset("test_inputs", data = test_inputs )
    f.create_dataset('test_outputs', data = test_outputs)
    f.create_dataset("test_input_times", data = test_input_times)
    f.create_dataset('test_output_times', data = test_output_times)
    f.create_dataset('original_test_inputs',data=original_test_inputs)
    f.create_dataset('original_test_outputs',data=original_test_outputs)    
    f.create_dataset("original_datas", data=original_datas)
#     f.create_dataset('original_inputs',data=original_inputs)
#     f.create_dataset('original_outputs',data=original_outputs)